# Import Libraries

In [1]:
import io
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from google.cloud import vision
from dateutil.parser import parse
import re

# Set Environment for API

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"rc_cred.json"

client = vision.ImageAnnotatorClient()

In [3]:
#Extract text from image using Google vision api

def detect(img):
    with io.open(img, 'rb') as image_file:
        content = image_file.read()
    
    image = vision.types.Image(content=content)
    response = client.document_text_detection(image=image)
    texts = response.text_annotations
    
    df = pd.DataFrame(columns = ["local","description"])
    
    for text in texts:
        df = df.append(
            dict(
            local = text.locale,
            description = text.description
            ),
            ignore_index = True
            
            )
        
    return df

In [4]:
#Match required string with detected string 

def func(string ,part):
    string = string.replace(" ","")
    part = part.replace(" ","")
    
    l1 = list(part)
    sl = list(string)
    
    for i in range(0,len(sl)-len(l1)+1):
        c = 0
        for j in range(0,len(l1)):
            if sl[i+j] == l1[j]:
                c += 1
        if c > (len(l1)*0.80):
            return True , i+j+1

In [5]:
#Check whether date or not

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [6]:
def is_reg(s):
    l = ["DL","HR"]
    flag = 0
    for i in l:
        if s.find(i) == -1:
            flag += 1
        else:
            strt= s.find(i)
            part = s[strt:]
            return part
            break
    if flag == 2:
        return s.split(":")[1]
            
    

In [7]:
def is_ch(s):
    s = s.replace(" ","")
    if func(s,"CHNO") != None:
        _ , strt = func(s,"CHNO")
        return s[strt:]
        
    if func(s,"Chassis No") != None:
        _ , strt = func(s,"Chassis No")
        return s[strt:]
        
        
    if func(s,"Chasis No") != None:
        _ , strt = func(s,"Chasis No")
        return s[strt:]

In [8]:
def is_eng(s):
    s = s.replace(" ","")
    if func(s,"ENO") != None:
        _ , strt = func(s,"ENO")
        return s[strt:]
        
    if func(s,"Engine No") != None or func(s,"Engine") != None:
        _ , strt = func(s,"Engine")
        if s[strt:].find("No") != -1:
            return s[strt+2:]
        else:
            return s[strt:]
        

In [9]:
def f(data):
    rno_info = "-"
    rdt_info = "-"
    ch_info = "-"
    e_info = "-"
    n_info = "-"
    mfg_info = "-"
    obj = data["description"][0].replace(".","").splitlines()
    for i in obj:
        
        
        if func(i,"REGNNO") or func(i,"Registration No"):
            if func(i,"Registration Valid Upto"):
                continue
            else:
                if rno_info == "-":
                    rno_info = is_reg(i)
        
        
        
        if func(i,"REGDT"):
            for j in i.split():
                    if is_date(j):
                        if rdt_info == "-":
                            rdt_info = j
            for k in obj:
                for m in k.split():
                    if is_date(m):
                        if m.isnumeric():
                            continue
                        else:
                            if rdt_info == "-":
                                rdt_info = m
                                break
                if rdt_info != "-":
                    break
                        
                        
        if func(i,"CHNO") or func(i,"Chassis No") or func(i,"Chasis No"):
            if i.find(":") != -1:
                if ch_info == "-":
                    ch_info = i.split(":")[1]
                for j in ch_info.split():
                    if j.isalpha():
                        end = ch_info.find(j)
                        if ch_info == "-":
                            ch_info = ch_info[:end]
                            break
            else:
                if ch_info == "-":
                    ch_info = is_ch(i)
                    for j in ch_info.split():
                        if j.isalpha():
                            end = ch_info.find(j)
                            ch_info = ch_info[:end]
                            break
                        
            
            
        if func(i,"ENO") or func(i,"Engine No") or func(i,"Engine"):
            if i.find(":") != -1:
                if e_info == "-":
                    e_info = i.split(":")[1]
            else:
                if e_info == "-":
                    e_info = is_eng(i)
            
            
            
        if func(i,"NAME") or func(i,"Name & Address") or func(i,"Owner's  Name"):
            l = []
            for j in i.split():
                if j.isupper():
                    if "NAME" in j:
                        continue
                    else:
                        if j.isalpha():
                            l.append(j)
                        else:
                            break
            n_info = " ".join(l)
                        
                        
                        
        if func(i,"MFGDT") or func(i,"Month/Year") or func(i,"Manufacturing"):
            for j in i.split():
                    if is_date(j):
                        if mfg_info == "-":
                            mfg_info = j
                            break
            for k in obj:
                for m in k.split():
                    if is_date(m):
                        if m.isnumeric():
                            continue
                        else:
                            if mfg_info == "-":
                                if rdt_info != "-":
                                    if rdt_info[-4:] >= m[-4:] and rdt_info != m:
                                        mfg_info = m
                                        break
                                        
                if mfg_info != "-":
                    break
    return rno_info ,rdt_info,ch_info ,e_info,n_info,mfg_info     

In [4]:
def main(img_path):
    columns = ["Registration No","Registration Date" , "chassis No" , "Engine No" , "Name" ,"Manufacturing Date"]
    data1 = detect(
    c1 , c2 , c3 , c4 , c5 , c6 = f(img_path))
    res = [c1,c2,c3,c4,c5,c6]
    df = pd.DataFrame(res , columns = columns)
    return df     

In [ ]:
#Pass image path in main()

file = main(/*img path*/)
file

In [278]:
file.to_csv("Output.csv")

In [279]:
pd.read_csv("Output.csv")

,Unnamed: 0,Registration No,Registration Date,chassis No,Engine No,Name,Manufacturing Date
0,0,DL2CAM0993,13/10/2010,MALBB51 BLAM217697,G4LAAM498098,MR KAMAL BHOLA,08/2010
1,1,HR42-A-4854,-,MAJEED81800628175,F10DN4452456,ASHWAN,23/01/2008
2,2,-,-,-,-,-,-
3,3,HR10-P-08403,-,1MALCG41GLAM255721*A,G4EB9M256677,SANDEEP LAKRATOR,-
4,4,-,-,-,-,-,-
5,5,DL3CAX6515,24/04/2009,607146KRZW00622,475IDT 14KRZPB8819,VIKAS ENGINEERING WORKS,-
6,6,DL3CAX6515,24/04/2009,607146KRZW00622,475IDT14KRZPB8819,VIKAS ENGINEERING WORKS,-
7,7,DL3CAX6515,24/04/2009,607146KRZW0062,475IDT14KPZPB8819,VIKAS ENGINEERING WORKS,-
8,8,DL2CAF6866,03/04/2006,MAJEEJ91 S00127444,LXIBS3,HIMANSHU,-
9,9,DL2CAF6866,03/04/2006,MAJEEJ91 S00127444,BALENO LXI BS3,HIMANSHU,-
